In [1]:
import ultralytics
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
model=ultralytics.YOLO("best.pt")

In [3]:
vitamines_file=pd.read_csv("food sheet.csv")

In [4]:
vitamines_dict = dict(zip(vitamines_file['food'], vitamines_file['index']))
vitamines_dict['orange']=vitamines_dict['orange ']
 

In [ ]:
vitamines_file

In [ ]:
contraindications_list=vitamines_file.contraindications.to_list()
vitamines_file

In [ ]:
path=r"C:\Users\User\Desktop\test images"
output_dir=r"C:\Users\User\Desktop\predicted_images"
all_images_predictions_info={}
contraindications="high cholesterol"

index=0
img_list = os.listdir(path)
num_images_to_predict=len(img_list)
for i in range(num_images_to_predict):
    one_image_predictions_info={}
    
    image_index=img_list[i]
    img_path = os.path.join(path, img_list[i])
    
    results = model.predict(img_path,save=True,conf=0.2)[0]
    predicted_class_names=[]
    names = model.names
    for r in results:
        for c in r.boxes.cls:
            predicted_class_names.append(names[int(c)])
    
    boxes = results.boxes.xyxy.cpu().numpy().tolist()  
    if len(predicted_class_names)>0:
        info_list=[]
        for j in range(len(predicted_class_names)):
            class_name = predicted_class_names[j]
            if class_name in vitamines_dict:
                info_list.append(vitamines_file.iloc[vitamines_dict.get(predicted_class_names[j])])
            else:
                pass
            
        one_image_predictions_info[image_index]=info_list
        
        all_images_predictions_info.update(one_image_predictions_info)
    
         
    if not boxes:
        print(f"No bounding boxes: {img_path}")
        continue
    
    if isinstance(boxes[0], float):  
        boxes = [boxes]
    
    image = cv2.imread(img_path)
    unique_class_names=[]
    for i,box in enumerate(boxes):
        if isinstance(box, float):
            print("Skipping float value. No bounding box found.")
            continue
        
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255,0, 0),1)  
        cv2.putText(image, predicted_class_names[i], (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        error_text_color=(0,0,255)   
        for g in range(len(one_image_predictions_info)):
            
            if one_image_predictions_info[image_index][g][-1]==contraindications:
                    text_color = (255, 255, 255)  
                    text_position = (20, image.shape[0] - 20)  
                    cv2.putText(image, f"{contraindications}", (int(100), int(300)),
                                cv2.FONT_HERSHEY_SIMPLEX, 1,error_text_color, 2)

        if predicted_class_names[i] not in unique_class_names:
            unique_class_names.append(predicted_class_names[i])
            
    
    
    horizontal_line=0
    for q in unique_class_names:
        print(q)
        text_position_bottom = (horizontal_line, image.shape[0] - 10) 
        cv2.putText(image, vitamines_file.loc[vitamines_dict.get(q),"vitamins / %DV"], 
                                                text_position_bottom, cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
        horizontal_line+=100
   


    
    index+=1
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output_path = os.path.join(output_dir, f"annotated_image_{index}.jpg")
    cv2.imwrite(output_path, image)

   
    plt.figure(figsize=(16,12),dpi=300)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()
    cv2.waitKey(0)


In [8]:
contraindications = "choresterol"
all_images_predictions_info = {}
mobile_camera_url = "http://192.168.8.219:4747/video"

cap = cv2.VideoCapture(mobile_camera_url)  
capture = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    capture += 1
    results = model.predict(frame, conf=0.05)[0]
    predicted_class_names = []
    names = model.names
    one_image_predictions_info = {}
    
    for r in results:
        for c in r.boxes.cls:
            predicted_class_names.append(names[int(c)])
    
    boxes = results.boxes.xyxy.cpu().numpy().tolist()  
    
    if len(predicted_class_names) > 0:
        info_list = []
        
        for j in range(len(predicted_class_names)):
            class_name = predicted_class_names[j]
            if class_name in vitamines_dict:
                info_list.append(vitamines_file.iloc[vitamines_dict.get(predicted_class_names[j])])
            else:
                pass
            
        one_image_predictions_info[f"VideoCapture {capture}"] = info_list
        all_images_predictions_info.update(one_image_predictions_info)
    
    image = frame
    unique_class_names = []
    
    for i, box in enumerate(boxes):
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 1)
        cv2.putText(image, predicted_class_names[i], (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        error_text_color = (0, 0, 255)
        for g in range(len(boxes)):
            print(g)
            if one_image_predictions_info[f"VideoCapture {capture}"][g][-1] == contraindications:
                text_color = (255, 255, 255)
                text_position = (0, image.shape[0] - 20)
                cv2.putText(image, f"{contraindications}", (int(100), int(300)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, error_text_color, 2)
                
        if predicted_class_names[i] not in unique_class_names:
            unique_class_names.append(predicted_class_names[i])
            
    horizontal_line = 0
    
    for q in unique_class_names:
        text_position_bottom = (horizontal_line, image.shape[0] - 10)
        cv2.putText(image, vitamines_file.loc[vitamines_dict.get(q), "vitamins / %DV"], 
                    text_position_bottom, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        horizontal_line += 250    
    
    cv2.imshow("Object Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 blueberry, 401.4ms
Speed: 8.5ms preprocess, 401.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



0


0: 480x640 1 blueberry, 341.0ms
Speed: 5.7ms preprocess, 341.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



0


0: 480x640 (no detections), 302.7ms
Speed: 3.3ms preprocess, 302.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blueberry, 299.0ms
Speed: 12.5ms preprocess, 299.0ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blueberry, 144.2ms
Speed: 7.5ms preprocess, 144.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



...

0: 480x640 (no detections), 106.9ms
Speed: 0.0ms preprocess, 106.9ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 fries, 116.8ms
Speed: 2.9ms preprocess, 116.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 fries, 116.5ms
Speed: 0.0ms preprocess, 116.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



0
0


0: 480x640 (no detections), 129.0ms
Speed: 0.0ms preprocess, 129.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 98.8ms
Speed: 4.8ms preprocess, 98.8ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 118.4ms
Speed: 4.5ms preprocess, 118.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 144.9ms
Speed: 0.0ms preprocess, 144.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 156.0ms
Speed: 0.0ms preprocess, 156.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 135.1ms
Speed: 0.0ms preprocess, 135.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 118.7ms
Speed: 0.5ms preprocess, 118.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 136.0ms
Speed: 5.4ms preproc

0



0: 480x640 (no detections), 168.6ms
Speed: 0.0ms preprocess, 168.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
all_images_predictions_info